In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

Using device: cuda


In [4]:
# Normalization values for CIFAR-10
mean = (0.4914, 0.4822, 0.4465)
std  = (0.2023, 0.1994, 0.2010)

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

trainset = torchvision.datasets.FashionMNIST(
    root='./data', train=True, download=True, transform=transform_train
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2
)

testset = torchvision.datasets.FashionMNIST(
    root='./data', train=False, download=True, transform=transform_test
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2
)

In [5]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels,
                               kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels, out_channels,
                               kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut (projection) if shape changes (stride != 1 or channels differ)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out


In [4]:
class ResNet56(nn.Module):
    def __init__(self, block=BasicBlock, num_classes=10):
        super(ResNet56, self).__init__()
        self.in_channels = 16

        # Initial conv: 3x3, 16 filters
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1,
                               padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        # Each layer: 9 blocks
        self.layer1 = self._make_layer(block, 16, blocks=9, stride=1)
        self.layer2 = self._make_layer(block, 32, blocks=9, stride=2)
        self.layer3 = self._make_layer(block, 64, blocks=9, stride=2)

        # Global average pool and final classification
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride):
        """Create a 'layer' consisting of `blocks` residual blocks."""
        strides = [stride] + [1]*(blocks - 1)
        layers = []
        for s in strides:
            layers.append(block(self.in_channels, out_channels, s))
            self.in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

In [5]:
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc

def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = 100.0 * correct / total
    return epoch_loss, epoch_acc


In [6]:
model = ResNet56().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

In [7]:
#transformations for metamorphic testing
transformations = [
    transforms.RandomRotation(degrees=10),  # Small rotation
    transforms.ColorJitter(brightness=0.2),  # Slight brightness change
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1))  # Small translation
]

In [8]:
import torch
import torch.nn.functional as F

def majority_vote_prediction(model, images, transformations, device):
    """
    Given a batch of images and a list of transformations,
    compute the model's predictions on:
        1) the original images,
        2) each transformed version,
    then take a majority vote across all predictions for each sample.
    
    Returns a tensor of shape [batch_size] with the final voted class.
    """
    model.eval()

    # 1) Original predictions
    with torch.no_grad():
        original_out = model(images)
        _, original_preds = torch.max(F.softmax(original_out, dim=1), dim=1)  # shape [batch_size]

    # 2) Collect predictions in a list
    all_preds = [original_preds]

    # 3) For each metamorphic transform
    for tf in transformations:
        # apply transform on CPU, then move back to device
        x_tf = tf(images.cpu()).to(device)

        with torch.no_grad():
            out_tf = model(x_tf)
            _, preds_tf = torch.max(F.softmax(out_tf, dim=1), dim=1)

        all_preds.append(preds_tf)

    # 4) Stack them into shape [num_transforms+1, batch_size]
    stacked_preds = torch.stack(all_preds, dim=0)

    # 5) Majority vote across the first dimension
    voted_preds = stacked_preds.mode(dim=0).values  # shape [batch_size]
    return voted_preds

In [9]:
from tqdm import tqdm

def test_with_majority_vote(model, loader, transformations, device):
    """
    Evaluates the model on 'loader' using majority-voted predictions (original + transforms).
    Prints per-batch accuracy and returns the final overall accuracy.
    """
    model.eval()
    correct = 0
    total = 0

    for i, (images, labels) in enumerate(tqdm(loader)):
        images, labels = images.to(device), labels.to(device)

        # Get majority-voted predictions
        voted_preds = majority_vote_prediction(model, images, transformations, device)

        # Compare to ground-truth
        batch_correct = (voted_preds == labels).sum().item()
        batch_size = labels.size(0)

        correct += batch_correct
        total += batch_size

        # Print a line for each batch
        print(f"Batch {i} | MajVot Accuracy: {100.0 * batch_correct / batch_size:.2f}% | Batch Size: {batch_size}")

    # Final overall accuracy
    overall_accuracy = 100.0 * correct / total
    print(f"\nFinal MajVot Accuracy (across all test samples): {overall_accuracy:.2f}%")
    return overall_accuracy

In [10]:
majvot_acc = test_with_majority_vote(model, testloader, transformations, device)
print(f"Majority Vote final accuracy: {majvot_acc:.2f}%")

  3%|██▌                                                                                   | 3/100 [00:00<00:25,  3.83it/s]

Batch 0 | MajVot Accuracy: 11.00% | Batch Size: 100
Batch 1 | MajVot Accuracy: 7.00% | Batch Size: 100
Batch 2 | MajVot Accuracy: 9.00% | Batch Size: 100
Batch 3 | MajVot Accuracy: 9.00% | Batch Size: 100


  8%|██████▉                                                                               | 8/100 [00:01<00:09, 10.19it/s]

Batch 4 | MajVot Accuracy: 11.00% | Batch Size: 100
Batch 5 | MajVot Accuracy: 10.00% | Batch Size: 100
Batch 6 | MajVot Accuracy: 8.00% | Batch Size: 100
Batch 7 | MajVot Accuracy: 13.00% | Batch Size: 100
Batch 8 | MajVot Accuracy: 17.00% | Batch Size: 100


 13%|███████████                                                                          | 13/100 [00:01<00:06, 14.46it/s]

...

Batch 89 | MajVot Accuracy: 9.00% | Batch Size: 100
Batch 90 | MajVot Accuracy: 9.00% | Batch Size: 100
Batch 91 | MajVot Accuracy: 8.00% | Batch Size: 100
Batch 92 | MajVot Accuracy: 18.00% | Batch Size: 100
Batch 93 | MajVot Accuracy: 15.00% | Batch Size: 100


 99%|████████████████████████████████████████████████████████████████████████████████████▏| 99/100 [00:05<00:00, 21.69it/s]

Batch 94 | MajVot Accuracy: 11.00% | Batch Size: 100
Batch 95 | MajVot Accuracy: 8.00% | Batch Size: 100
Batch 96 | MajVot Accuracy: 9.00% | Batch Size: 100
Batch 97 | MajVot Accuracy: 9.00% | Batch Size: 100
Batch 98 | MajVot Accuracy: 9.00% | Batch Size: 100


100%|████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:05<00:00, 17.44it/s]

Batch 99 | MajVot Accuracy: 13.00% | Batch Size: 100

Final MajVot Accuracy (across all test samples): 10.00%
Majority Vote final accuracy: 10.00%
